In [ ]:
from typing import List, Dict
from sagemaker import Session
import boto3
import json

In [ ]:
# Need update to GNU C++ compiler version
!apt-get update && apt-get install -y build-essential --quiet
!pip install shap --quiet
# Installs dependencies reuqired for chromadb memory vectordb and embedding library
!pip install chromadb tiktoken langchain --quiet


# Building Foundation Model based Applications

Working with LLMs to provide you with advanced reasoning and routing capabilities is easy to get started with. After all the models understand human level instructions, and provide formattable string outputs as a result. 

Yet, when you are looking to develop production ready applications you will require robust data integrations to provide input to your model, you want to solve the alignment problem with LLMs, tune the behavior to your specific corporate governance and brand messaging.

Complex workflows will involve multiple stages to create intermediate results. These stages require the model to switch roles, or might involve optimized task models to be more efficient, or even fine-tuned further on the specific tasks. 

All of this creates complexity when creating and maintaining your LLM based applications in practice.



In [ ]:
#load stored variables from previous notebook
%store -r

# Initialize key environment variables
sagemaker_session = Session()
aws_role = sagemaker_session.get_caller_identity_arn()
aws_region = boto3.Session().region_name
sm_client = boto3.client("sagemaker", aws_region)
model_version = "*"

print(falcon_inference_model)
print(embedding_model)
try:
    print(ai21_inference_model)
except:
    print("You did not load the AI21 Jurassic Grande model and will not be able to select it for this lab")

### Registering your Third-Party API key (PURELY OPTIONAL!)
We will be running a section on the ChatModel API exposed by a series of API endpoint providers such as OpenAI, Anthropic, Google Vertex. As this is currently not supported by the SageMaker deployed models, you can choose to experimment with at your own costs if you register for an OpenAI key, or you have previous access to Anthropic (as they are currently not accepting new registrations)

In [ ]:
openai_api_key=""
anthropic_api_key=""

In [ ]:
# Installing reuqired dependencies for third party Foundation APIs
import sys
if openai_api_key:
    !pip install openai --quiet
if anthropic_api_key:
    !pip install anthropic --quiet


#### Load Widgets used across the notebook

In [ ]:
from ipywidgets import Select, Text

# This creates the widgets used across the notebook for easier configuration
model_selections = [
    'SageMaker-Falcon40B', 
    # 'AI21-Jurrasic-Grande'
]
# Subset based on available ApiKeys
if openai_api_key:
    model_selections.append('OpenAI')
if anthropic_api_key:
    model_selections.append('Anthropic-Claude')

model_selection_widget = Select(
    options=model_selections
)

In [ ]:
chat_model_selections = []
if openai_api_key:
    chat_model_selections.append('ChatOpenAI')
if anthropic_api_key:
    chat_model_selections.append('ChatAnthropic')

chat_model_selection_widget = Select(
    options=chat_model_selections
)

# Using the power of LangChain
Recently the community unified their efforts on a high-level Framework to ease the development of foundation model based applications.
LangChain was developed to ease the integration of models deployed, or used over proprietary APIs. It lets you easily integrate models into your application, manage the templates for prompts to tune your model behaviour, provide IO, add memory and chain multiple reasoning and action steps. 

### What is LangChain
LangChain is a framework for developing applications powered by language models.

It helps us with:
1. **Integration** - Bring external data, such files, databases, webcontent, API data to your application
2. **Coordination** - Develop reusable, modularized pipelines to execute complex workflows 
3. **Agency** - Enable your LLM to interact with it's environmetn via decision making

## Benefits of using the Framework
1. Components - LangChain makes it easy to swap out abstractions and components necessary to work with language models.

2. Customized Chains - LangChain provides out of the box support for using and customizing 'chains' - a series of actions strung together.

3. Speed 🚢 - This team ships insanely fast. You'll be up to date with the latest LLM features.

4. Community 👥 - Wonderful discord and community support, meet ups, hackathons, etc.


## Connecting your model on AWS
To work with your models on AWS you can use either an integration with the SageMaker endpoint, or in the future directly talk to the Bedrock API. For now, let's look at how to work with a custom SageMaker Model Endpoint.

### Configuring the model parameters

In [ ]:
# Model configuration for falcon 40b instruct
falcon_parameters = {
    "max_new_tokens": 200,
    "max_length": 1024,
    # "num_return_sequences": 1,
    "top_k": 1,
    # "top_p": 0.50,
    "do_sample": True,
    "temperature": 0.1,
    "return_full_text": False,
    "include_prompt_in_result": False,
}

# Configuration for Jurassic Grande Model
j2_grande_parameters = {
    "max_new_tokens": 200,
    "max_length": 1024,
    # "num_return_sequences": 1,
    "top_k": 1,
    # "top_p": 0.50,
    "do_sample": True,
    "temperature": 0.1,
    "return_full_text": False,
    "include_prompt_in_result": False,
}

In [ ]:
import json
from langchain.llms.sagemaker_endpoint import LLMContentHandler, SagemakerEndpoint
from langchain.embeddings.sagemaker_endpoint import EmbeddingsContentHandler
from langchain.embeddings import SagemakerEndpointEmbeddings

class InferenceContentHandler(LLMContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        input_str = json.dumps({"inputs": prompt, "parameters": model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        return response_json[0]["generated_text"]

class EmbeddingsContentHandler(EmbeddingsContentHandler):
    content_type = "application/json"
    accepts = "application/json"

    def transform_input(self, prompt: str, model_kwargs={}) -> bytes:
        input_str = json.dumps({"text_inputs": prompt, **model_kwargs})
        return input_str.encode("utf-8")

    def transform_output(self, output: bytes) -> str:
        response_json = json.loads(output.read().decode("utf-8"))
        embeddings = response_json["embedding"]
        return embeddings

    
inference_content_handler = InferenceContentHandler()
embeddings_content_handler = EmbeddingsContentHandler()

# Instantiate all available models
falcon_sm_llm = SagemakerEndpoint(
    endpoint_name=_MODEL_CONFIG_[falcon_inference_model]['endpoint_name'],
    region_name=aws_region,
    model_kwargs=falcon_parameters,
    content_handler=inference_content_handler,
    )
print(f"Loaded model endpoint: {falcon_inference_model}")
embedding_sm_llm = SagemakerEndpointEmbeddings(
    endpoint_name=_MODEL_CONFIG_[embedding_model]['endpoint_name'],
    region_name=aws_region,
    content_handler=embeddings_content_handler,
    )
print(f"Loaded model endpoint: {embedding_model}")

# Optional load
# j2_grande_sm_llm = SagemakerEndpoint(
#     endpoint_name=_MODEL_CONFIG_[ai21_inference_model]['endpoint_name'],
#     region_name=aws_region,
#     model_kwargs=j2_grande_parameters,
#     content_handler=inference_content_handler,
#     )
# print(f"Loaded model endpoint: {ai21_inference_model}")



### Instantiate an proprietary API endpoint such as OpenAI
To baseline the performance of a model on an industry wide benchmark you can also execute your code against proprietary services. This is enabled through custom connector classes that rely only on the configuration of your api_keys.

In [ ]:
# Connecting to Third-party endpoints using provided API keys 
from langchain import OpenAI

if openai_api_key:
    openai_llm = OpenAI(openai_api_key=openai_api_key)
    print("(success) - Successfully connected to OpenAI")
else:
    print("(failure) - You have not provided an OpenAPI key, and you won't have access to work with the model in this notebook")

# Work with Anthropic
from langchain import Anthropic

if anthropic_api_key:
    anthropic_llm = Anthropic(anthropic_api_key=anthropic_api_key)
    print("(success) - Successfully connected to Anthropic")
else:
    print("(failure) - You have not provided an AnthropicAPI key, and you won't have access to work with the model in this notebook")

## Select your model
To showcase you how different the models behave to prompting you can choose to select between an OpenSource Leaderboard model `Falcon-40B-Instruct Model` and `OpenAIs Davinci Model` 

In [ ]:
model_selection_widget

In [ ]:
match model_selection_widget.value:
    case "SageMaker-Falcon40B":
        llm = falcon_sm_llm
    case "AI21-Jurrasic-Grande":
        llm = j2_grande_sm_llm
    case "OpenAI":
        llm = openai_llm
        
print(f"Activated {model_selection_widget.value}")

In [ ]:
print(llm("What day comes after Friday").strip())

# Creating a basic langchain application

Every LangChain application centers around your LLM model. This can be either a deployed inference endpoint, or a managed service (Bedrock, OpenAI API). The framework provides a series of out of the box integrations in the `llms` module and can be easily expanded to your use case. 



## Models

A model takes a series of messages and returns a message as output

You can choose between:
1. **LanguageModel** Takes text and returns text
2. **Chat Model** Takes a series of messages and returns a message output
3. **Embedding Models** Transform your text into a latent space vector to power similarity search

### Language Models
A wrapper around a typical text input, text output interaction with the model. No structure is expected, and no structure is maintained. Good starting point for many non-chat applications. 

Now that we established our connection, we can query the model by sending it instructions as text.

In [ ]:
text = "Give me 10 names for a template factory library for prompt engineering. Ensure to create the required number of examples. Only provide the items of the list"
print(llm(text))

In [ ]:
complex_prompt = """
Create a list of services a company named {prompt} could sell.
"""

In [ ]:
print(llm(complex_prompt.format(prompt="FactoryBot")))

And we can use vanilla string formatting to integrate information into our models. This allows us to pass information in a structed manner into the model, masking the general nature of the model. This allows to create all the common products you see being built natively on LLMs. 

In [ ]:
architect_prompt = """
Play the role of a solution architect experienced with AWS. You are analysing customer requirements to create
well-architected solution architectures that you present to the customer. You are detailled, kind and
focussed. Given the following context

Context:
#System Requirements:
{requirements}
#Scale:
{scale}
#Features:
{features}
Describe an architecture on AWS in technical detail.
"""

In [ ]:
prompt = architect_prompt.format(
    requirements="A website for my foodstore", 
    scale="Must handle 10k requests per second in peak. Must be globally available. Must be reponsive and fast", 
    features="Landing page describing our product. About page describing the company. Career page describing open positions."
)
print(prompt)

In [ ]:
print(llm(prompt))

This works but can get a bit clunky when you try to scale it out to more complex use cases. The next type of model wrapper provides a solution to this problem

### Chat Models
These models structure their input and outputs with Schemata that enable you to reason about the expected input and output process. This helps to build more complex designs by seperating the inputs used to provide the model with its role instruction, the query and the context to the query. 

Currently this is only implemented for API based models such as ChatGTP and Anthropic.

This section is OPTIONAL, as you will have to have your own ChatAntrophic API key to follow along. Currently registration for API keys is closed as they roll out the service. If you do not have a key yet, just read through the outputs of the notebook for reference. 


In [ ]:
chat_model_selection_widget

In [ ]:
# Load selected ChatModel Endpoint
from langchain.chat_models import ChatOpenAI, ChatAnthropic
match chat_model_selection_widget.value:
    case "ChatOpenAI":
        chat_llm = ChatOpenAI(openai_api_key=openai_api_key) 
    case "OpenAI":
        llm = ChatAnthropic(anthropic_api_key=anthropic_api_key)
        
print(f"Activated {chat_model_selection_widget.value} as chat_llm")

In [ ]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage
response = chat_llm([
    SystemMessage(content="You are an unhelpful AI bot that makes jokes at whatever the user says."),
    HumanMessage(content="I would like to go to New York, how should i do this?"),
    AIMessage(content="???")
])
print(response.content)

## Schemata

We see that ChatModels use typed classes to structure inputs. This is an example of a LangChain `Schema`, but its just one of many. 

LangChain currently provides the following schemata:

* **Text** The primary interface to interact with a model (used with LanguageModels
* **ChatMessages** What you saw we defined up with the ChatModel
* **Examples** Input/output pairs acting as context for fine tuning model behavior in n-shot learning
* **Document** Piece of unstructured data holding data as content and metadata for retrieval in context

### ChatMessages Schema
The primary interface through which end users interact with these is a chat interface. For this reason, some model providers even started providing access to the underlying API in a way that expects chat messages.

In [ ]:
from langchain.schema import HumanMessage, SystemMessage, AIMessage

hum_msg = HumanMessage(content='inputs send to the model by the user', additional_kwargs={}, example=True)
hum_msg

In [ ]:
sys_msg = SystemMessage(content='Instructions to the model', additional_kwargs={})
sys_msg

In [ ]:
ai_msg = AIMessage(content='Context answer providing further input to the model', additional_kwargs={})
ai_msg

This structure allows us to simply pass multiple requests into a model for batch processing, making application integration easier

In [ ]:
# Generate completions for multiple sets of messages
batch_messages = [
    [   SystemMessage(content="You are a helpful assistant that translates English to German."),
        HumanMessage(content="What a wonderful day we had at the beach this late summer.")
    ],
    [
        SystemMessage(content="You are a helpful assistant that translates English to malay."),
        HumanMessage(content="What a wonderful day we had at the beach this late summer.")
    ]
]

In [ ]:
chat_llm.generate(batch_messages)

## EXERCISE 1
We are working to enable our marketing team to provide customized sales emails at scale. You are asked to create to engineer a prompt for a custom marketing email copy creation pipeline. 

You will be given the following inputs that are collected on the users in your database:
* Name 
* Age
* Interest (List of strings)

You will also be given a recommended product to personalize-recommend to the user
* Product described as a dictionary of attributes (document from DB)


Work to complete the function below:

In [ ]:
from typing import List
#TODO Rewrite for callcenter use case

# Complete the function 
def create_email_copy(name: str, age: int, interests: List[str], product: dict) -> str:
    """
    The email should be personalized, be age appropriate, target the interests 
    of the person and market the product you are selling. 

    Fill in this template using string formatting and a combination of the prompt
    engineering techniques you have learned previously. 
    """
    pass

In [ ]:
# Define the product you are selling. Play with the level of detail

_product = {}

In [ ]:
# Define a set of users to generate eamils for
users = [
    {
    "name": "",
    "age": 0,
    "intesrests": [],
    "product": _product
    }
]

In [ ]:
# Test your marketing output
for user in users:
    print("\n\n")
    print(llm(create_email_copy(user)))


# Prompt templates 

When building more complex scenarios, managing the parameters placed into the templates can be too complex for simple string injection methods. Eventually you want to describe the interface in a more programmatic way. Here the `PromptTemplate` helps to define verified input variables to be utilized in the format string.

### The PromptTemplate class 

Let's structure our architecture template to make it reusable in our architecture.

In [ ]:
print(architect_prompt)

In [ ]:
from langchain.prompts import PromptTemplate

# First we can define an exposed parameter interface to the format string
prompt = PromptTemplate(
    input_variables=["requirements", "scale", "features"],
    template=architect_prompt,
)

The template can be asked to format itself, returning the compiled format string for review.

In [ ]:
final_prompt = architect_prompt.format(
    requirements="External facing web application written in Javascript, global deployment",
    scale="Average of 500 requests per minute, scale events up to 3000 requests per second",
    features="Mobile website, desktop version, javascript"
)
print(final_prompt)

In [ ]:
llm(final_prompt)

Having a string output is nice and dandy, but what if we want to create structure returns for further use in our applications. For example, how would we continue working with an extracted set of attributes from a text in a parser scenario? 

Is a string good enough, or would we rather want to return a named tuple, dict or list of class instances from the model?

In [ ]:
template="""
    You identify named entities in the text and extract relations amongst them. 
    You do not answer questions, and you do not ask questions.
    It is very important to extract all references you find. Each referenc contains (Subject, relationship, value). 
    Do not skip any in your output.
    {format_instructions}

    # Examples:
    The Dow Jones closed with a plus of 1456 points // [("Dow Jones", "closed", "1456 points")]
    The ./ is a relative path and assumes that you are currently in your virtual environment directory // [("./", "is", "relative path")]
    Q: {text} // 
    """
reference_template = PromptTemplate(
    template=template,
    input_variables=['text', 'format_instructions'],
)

In [ ]:
text = "Putting in effort means going beyond what’s required to solve problems, even when you aren’t asked to — on top of your job’s normal responsibilities, Cuban said. You take the initiative, and exhaust every possible option to find answers."
llm(reference_template.format(text=text, format_instructions=""))

### Output parsers
When we need to validate the output of a model to a given prompt or return a value as a program language object instead of a plain string we can use output parsers.

The class implements a dual interface:
1. It standardizes prompt engineering to align the output of the model to the required format
2. It parses the resulting model output into the desired language primitive (list, dict, object)

The parser classes available allow to create `pydantic` schema objects that can integrate validation steps.

#### List parser
Like the most common scenario is to handle a return of multiple items in a list. We want to prompt the model into returning the elements in a nicely formatted structure. Typically this will be a CSV format. 

In [ ]:
# List generating prompt
topic_recommender_prompt="List {number} topics to write on blog posts about {topic}"

recommend_topic_prompt = PromptTemplate(
    template=topic_recommender_prompt,
    input_variables=['topic', 'number']
)

llm(recommend_topic_prompt.format(topic="Python", number=10))

In [ ]:
from langchain.output_parsers import CommaSeparatedListOutputParser
parsed_recommender_prompt = topic_recommender_prompt + "\n{format_instructions}"

parser = CommaSeparatedListOutputParser()

parsed_recommender_template = PromptTemplate(
    template=parsed_recommender_prompt,
    input_variables=['topic', 'number'],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)


In [ ]:
gen_prompt = parsed_recommender_template.format(topic='Generative AI', number=10)
print(gen_prompt)

In [ ]:
output = llm(gen_prompt)
output

#### Custom parser
For any entity model that relies on structure information to be returned we will have to implement a custom model based on the `BaseModel` from the `pydantic` library.

The model class describes the expected schema and an optional set of validation functions to ensure the accepted values are properly configured.

In [ ]:
from pydantic import BaseModel, Field, validator
from langchain.output_parsers import PydanticOutputParser
from typing import List

# Define the target structure
class Entity(BaseModel):
    subject: str = Field(description="subject of the relation")
    object: str = Field(description="object of the relation")
    relation: str = Field(description="relation between subject and object")

parser = PydanticOutputParser(pydantic_object=Entity)


In [ ]:
# Let's go back to our entity extraction use case
extraction_template = """
You are extracting relations between entities in a text.
You extract them in the format (Subject, Predicate, Object).
All relations are in present tense.
{format_instructions}
Input: The play followed the story of Edalaine. She was a young woman.The play was written by Edgar Allan Poe.
[('play', 'about', 'Edalaine'), ('Edalaine', 'age', 'young'), ('Edalaine', 'gender', 'female'), ('play', 'written by', 'Edgar Allan Poe')]

{text}
"""

In [ ]:
parsed_template = PromptTemplate(
    template = extraction_template,
    input_variables=['text'],
    partial_variables={"format_instructions": parser.get_format_instructions()}
)

In [ ]:
display(parsed_template.to_json())

In [ ]:
# The model we defined is parsed into the following context template format
print(parsed_template.to_json()['kwargs']['partial_variables']['format_instructions'])

In [ ]:
responses = llm(parsed_template.format(text=text))

In [ ]:
responses

The template is not yet tuned to identify the correct subjects to extract information for. We can continue to refine the template to hit our expected target. Yet, we see that the model is producing the target format, so we can continue to parse the output.

In [ ]:
entity = parser.parse(responses)
print(type(entity))
print(entity)

##  Example
These can be inputs/outputs for a model or for a chain. Both types of examples serve a different purpose. Examples for a model can be used to finetune a model. Examples for a chain can be used to evaluate the end-to-end chain, or maybe even train a model to replace that whole chain.

In [ ]:
# Examples are structured as Q/A pairs. Let's create a list of fewshot examples for us to explore
examples = [
  {
    "question": "Who lived longer, Muhammad Ali or Alan Turing?",
    "answer": 
    """
        Are follow up questions needed here: Yes.
        Follow up: How old was Muhammad Ali when he died?
        Intermediate answer: Muhammad Ali was 74 years old when he died.
        Follow up: How old was Alan Turing when he died?
        Intermediate answer: Alan Turing was 41 years old when he died.
        So the final answer is: Muhammad Ali
    """
  },
  {
    "question": "When was the founder of craigslist born?",
    "answer": 
    """
        Are follow up questions needed here: Yes.
        Follow up: Who was the founder of craigslist?
        Intermediate answer: Craigslist was founded by Craig Newmark.
        Follow up: When was Craig Newmark born?
        Intermediate answer: Craig Newmark was born on December 6, 1952.
        So the final answer is: December 6, 1952
    """
  }
]

In [ ]:
# Then we use a formatter to parse the examples into string inputs to our template
example_prompt = PromptTemplate(input_variables=['question', 'answer'], template="Question: {question}\n{answer}")

In [ ]:
print(example_prompt.format(**examples[0]))

In [ ]:
from langchain import FewShotPromptTemplate
# Feed the examples and formatter to FewShotPromptTemplate

prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=['input']
)
print(prompt.format(input="Who was the father of Mary Ball Washington?"))

## Using an example selector
If you provide a full set of examples that cover various different topics in depth the lenght of the context can overrun the memory allocation of your model endpoint. 

Example selectors help to pass a subset of the examples that are relevant to the specific question at hand instead of passing the full examples.

The example selector utilizes a similarity score across the embedded question and example pairs. We will cover embeddings in detail in Lab2.

In [ ]:
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings


example_selector = SemanticSimilarityExampleSelector.from_examples(
    examples=examples,
    embeddings=embedding_sm_llm,
    vectorstore=Chroma,
    k=1
)

# Select the most similar example to the input.
question = "Who was the father of Mary Ball Washington?"
selected_examples = example_selector.select_examples({"question": question})
print(f"Examples most similar to the input: {question}")
for example in selected_examples:
    print("\n")
    for k, v in example.items():
        print(f"{k}: {v}")

We can use the `example_selector` instead of a hardcoded list of examples. Under the hood this similarity search utilizes embeddings and vector stores. m

In [ ]:
prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=example_prompt,
    suffix="Question: {input}",
    input_variables=['input']
)
print(prompt.format(input="Who was the father of Mary Ball Washington?"))

# Langchaining 

More complex workflows will involve the abbility of the model to react to specific subsets of tasks with specialized sequence of behaviors. We would capture these subsections of our application into modules called `chains`. 

Each chain structures the interaction with a model through specialized prompts, optional examples and optional output parsers. 

The langchain chains module contains the classes to help us easily create specialized sequences of chains that can receive inputs from previous model inferences as structured outputs. 

### Basic LLMChain

Let's play through the example of creating a product that allows users to generate a full set of marketing materials.

1. Generate name proposals for a company based on intended product to be sold
2. Generate a marketing slogan based on company values provided
3. Create a marketing template for email communication for the new company launch



### Generating the company name

In [ ]:
from langchain.chains import LLMChain

prompt = PromptTemplate(
    input_variables=["product"],
    template="""
    You are a helpful marketing assistant that creates a marketable company name for a company selling {product} 
    Answer with a single name only no comments or discussion.
    """
)

In [ ]:
# The LLMChain picks up the work for us to prompt the model with based on the template
company_name_chain = LLMChain(llm=llm, prompt=prompt, output_key="company_name")

In [ ]:
# We pass all variables required in the PromptTemplate directly into the chain
product = "colorful socks"
company_name = company_name_chain.run(product)
print(company_name)

A chain can generate batch predictions to answer multiple questions at a time using the `generate` method

In [ ]:
qs = [
    {'product': "Kids kites"},
    {'product': "Running shoes"},
    {'product': "Tennis sports wear"},
]

company_name_generated = company_name_chain.generate(qs)

In [ ]:
for response in company_name_generated.generations:
    print(response[0].text)

Let's consider a more realistic problem to solve with LLMs that we can not easily solve with a simngle prompt. Here we see the power of chaining a series of steps conducted by an agent using serialized IO of the results.

In [ ]:
services_prompt = PromptTemplate(
    input_variables=["company_name", "product"],
    template="Create a list of services and products that {company_name} can develop around {product}. The list must be commma seperated such as (abc, def, ghi, jkl)",
)
services_chain = LLMChain(
    llm=llm,
    prompt=services_prompt,
    output_key="services"
)

In [ ]:
# Step 1: We get the recommended list of services 
services = services_chain.run(product=product, company_name=company_name)
print(services)

In [ ]:
# Next let's create recommendations for the company slogan
slogan_prompt = PromptTemplate(
    input_variables=['product','company_name','services'],
    template="""
    Context:
    You are desigining a corporate identity for {company_name} selling {product}
    The company is providing {services}. 
    Create a slogan for the company that is unique and memorable.
    """
)
# Slogan chain
slogan_chain = LLMChain(
    llm=llm,
    prompt=slogan_prompt,
    output_key="slogan"
)

In [ ]:
slogan = slogan_chain.run(company_name=company_name, product=product, services=services)
print(slogan)

### Chaining the chains 
Now that we defined the stages of our chain, identified the inputs and outputs we require and engineered our prompt templates to get the desired outputs, we can build an integrated end-to-end chain.

In [ ]:
from langchain.chains import SequentialChain

marketing_chain = SequentialChain(
    chains=[company_name_chain, services_chain, slogan_chain],
    input_variables=['product'],
    output_variables=['company_name', 'services', 'slogan']
    
)

In [ ]:
marketing_chain("Kites")

#### What we could improve
You will likely see that your models won't always comply with your formatting rules you try to put into their context instructions. You can further expand on this chain design by implementing the output parsers between the stages to clean up the intermediary results. 

# EXERCISE 2
Let's bring it all together to create a custom LangChain that can build up a cooking article for our new online magazine. 

We will develop a series of chains to expand on a set of initial travel destinations to cover. 

Each article will contain:
* A overview section of the travel destination
* A list of things to do in the region
* A recipe for a famous local dish (description, list of ingredients, step by step instruction)

### Target
Your system should create the articles based on the following inputs:

* Travel destination (City, Country)

### Focus on this exercise
* Apply modularity to the steps in your workflow
* Consider quality gates when chaining values across steps
* Think about suitable prompt engineering methods to improve your results

In [ ]:
# Let's start by creating the subchains

# 1. Create the destination_overview chain


# 2. Create the recommended_activities chain


# 3. Create the famous_local_dish recommender 


Each of the chains should be constructed for a subset of the workflow that is self contained. Extending too many tasks at once can overload the model and lead to poor results. 

In [ ]:
#  Now unifiy the elements and ensure the information flow properly

In [ ]:
# Create a list of locations to cover in your reports
location_list = []

In [ ]:
# Execute the chain against the location_list
result = ...

## Summary
We have seen how LangChain provides us with a series of abstractions on the core building blocks of LLM based applications. 

We have learned to connect our models, create custom Templates, use Schemata to structure our message flow, chain a series of steps and structure the models outputs to further pass into downstream systems. 

Each component of the system is under continued development and likely you will see the library change as it continues to mature. 

Still, we believe LangChain to be a useful abstraction to develop faster and build a more sustainable code base. 